In [1]:
import tidypolars as tp
import polars as pl
from polars import col
import pandas as pd
import numpy as np
from timeit import timeit
import string

np.random.seed(123)

letters = np.asarray(list(string.ascii_lowercase))
letters = np.random.choice(letters, 10)

data_size = 1000000
# data_size = 1000
rand_string = np.random.choice(letters, data_size)
for i in range(3):
    rand_string = np.char.add(rand_string, np.random.choice(letters, data_size))

tidypolars_df = tp.Tibble(
    a = np.random.choice(np.arange(20), data_size),
    b = np.random.choice(np.arange(20), data_size),
    c = np.random.choice(rand_string, data_size),
    d = np.random.choice(rand_string, data_size)
)
polars_df = tidypolars_df.clone().to_polars()
pandas_df = polars_df.to_pandas()

In [2]:
def benchmark_me(d, num_tests):
    for i, (key, value) in enumerate(d.items()):
        if i == 0:
            out = tp.Tibble({key: [timeit(value, number = num_tests)]})
        else:
            step = tp.Tibble({key: [timeit(value, number = num_tests)]})
            out = out.bind_cols(step)
    return out.mutate((tp.col_everything() * 1000).round(3).cast(pl.Float64))

In [3]:
median_x = tidypolars_df.summarize(avg = col('a').mean()).pull('avg')[0]

summarize_funcs = { 
    'arrange' : dict(
        tidypolars = lambda: tidypolars_df.arrange('a'),
        polars = lambda: polars_df.sort(col('a')), 
        pandas = lambda: pandas_df.sort_values(by=['a'])
    ), 
    'case_when' : dict(
        tidypolars = lambda: tidypolars_df.mutate(x_case = tp.case_when(col('a') < median_x).then(1).when(col('a') >= median_x).then(2).otherwise(3)),
        polars = lambda: polars_df.with_columns(pl.when(col('a') < median_x).then(1).when(col('a') >= median_x).then(2).otherwise(3)), 
        pandas = lambda: pandas_df.assign(x_case = lambda x : np.where(x.a > median_x , 1, (np.where(x.a >= median_x, 2, 3))))
    ), 
    'distinct': dict(
        tidypolars = lambda: tidypolars_df.distinct('a'),
        polars = lambda: polars_df.select('a').drop_duplicates(),
        pandas = lambda: pandas_df.a.drop_duplicates()
    ), 
    'filter' : dict(
        tidypolars = lambda: tidypolars_df.filter(col('a') <= 7, col('c') == 'brkc'),
        polars = lambda: polars_df.filter((col('a') <= 7) & (col('c') == 'brkc')),
        pandas = lambda: pandas_df[(pandas_df.a <= 7) & (pandas_df.c == 'brkc')]
    ),
    'full_join' : dict(
        tidypolars = lambda: tidypolars_df.full_join(tidypolars_df.head(1000), on = 'c'),
        polars = lambda: polars_df.join(polars_df.slice(0, 1000),  on = 'c', how = 'outer'), 
        pandas = lambda: pandas_df.set_index('c').join(pandas_df.head(1000).set_index('c'), on = 'c', how='outer', rsuffix='right_')
    ), 
    'inner_join' : dict(
        tidypolars = lambda: tidypolars_df.inner_join(tidypolars_df.head(1000), on = 'c'),
        polars = lambda: polars_df.join(polars_df.slice(0, 1000),  on = 'c', how = 'inner'), 
        pandas = lambda: pandas_df.set_index('c').join(pandas_df.head(1000).set_index('c'), on = 'c', how='inner', rsuffix='right_')
    ), 
    'left_join' : dict(
        tidypolars = lambda: tidypolars_df.left_join(tidypolars_df.head(1000), on = 'c'),
        polars = lambda: polars_df.join(polars_df.slice(0, 1000),  on = 'c', how = 'left'), 
        pandas = lambda: pandas_df.set_index('c').join(pandas_df.head(1000).set_index('c'), on = 'c', how='left', rsuffix='right_')
    ), 
    'mutate' : dict(
        tidypolars = lambda: tidypolars_df.mutate(double_a = col('a') * 2, a_plus_b = col('a') + col('b')),
        polars = lambda: polars_df.with_columns([(col('a') * 2).alias('double_a'), (col('a') + col('b')).alias('a_plus_b')]), 
        pandas = lambda: pandas_df.assign(double_a = lambda x: x.a * 2, a_plus_b = lambda x : x.a + x.b)
        ), 
    'pivot_wider' : dict(
        tidypolars = lambda: tidypolars_df.head(5000).select(['c', 'd', 'a']).pivot_wider(names_from = 'c', values_from = 'a', values_fill = 0, values_fn = 'sum'),
        polars = lambda: polars_df.slice(0, 5000).select(['c', 'd', 'a']).groupby('c').pivot(pivot_column='c', values_column='a').sum().fill_null(0), 
        pandas = lambda: pandas_df.head(5000).groupby(['c', 'd'], as_index = False)['a'].sum().pivot(index = "d", columns = "c", values = "a").fillna(0)
    ), 
    'summarize': dict(
        tidypolars = lambda: tidypolars_df.summarize(x = col('a').median(), by = 'c'),
        polars = lambda: polars_df.groupby('c').agg(col('a').median().alias('x')),
        pandas = lambda: pandas_df.groupby('c', as_index = False)['a'].median()
    ) 
}


# Benchmark Results

In [5]:
for i, (key, value) in enumerate(summarize_funcs.items()):
    value = benchmark_me(value, num_tests = 5).mutate(func_tested = tp.lit(key)).relocate('func_tested')
    if i == 0:
        bench_df = value
    else:
        bench_df = bench_df.bind_rows(value)
        
bench_df.arrange('func_tested')

func_tested,tidypolars,polars,pandas
str,f64,f64,f64
"""arrange""",183.835,177.331,468.075
"""case_when""",79.826,77.972,104.921
"""distinct""",17.666,17.616,27.404
"""filter""",29.96,32.027,228.179
"""full_join""",226.388,229,807.002
"""inner_join""",48.935,49.238,419.765
"""left_join""",111.291,112.687,812.486
"""mutate""",8.7,7.449,70.117
"""pivot_wider""",5492.357,5370.443,412.117
